<a href="https://colab.research.google.com/github/DylanCTY/TextAnalytics_LearningSpace/blob/main/IB9CW0_5504008_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Install Ollama

In [1]:
# Install Ollama v0.1.30
!curl https://ollama.ai/install.sh | sed 's#https://ollama.ai/download#https://github.com/jmorganca/ollama/releases/download/v0.1.30#' | sh

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0>>> Downloading ollama...
100 10941    0 10941    0     0  23741      0 --:--:-- --:--:-- --:--:-- 23784
############################################################################################# 100.0%
>>> Installing ollama to /usr/local/bin...
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


Get Ollama running in the background of the Colab

In [2]:
# Setup the model as a global variable
OLLAMA_MODEL='phi:latest'

# Add the model to the environment of the operating system
import os
os.environ['OLLAMA_MODEL'] = OLLAMA_MODEL
!echo $OLLAMA_MODEL # print the global variable to check it saved

import subprocess
import time

# Start ollama on the server ("serve")
command = "nohup ollama serve&" # "nohup" and "&" means run in the background

# Use subprocess.Popen to run the command
process = subprocess.Popen(command,
                            shell=True,
                            stdout=subprocess.PIPE,
                            stderr=subprocess.PIPE)

print("Process ID:", process.pid) # print the process ID
time.sleep(5)  # Makes Python wait for 5 seconds

!ollama -v # print the Ollama version number as a check

phi:latest
Process ID: 1258
ollama version is 0.1.39


Query the model and get it to generate some text about Game of Thrones?

In [3]:
# Query the model via the command line
# First time running it will "pull" (import) the model
!ollama run $OLLAMA_MODEL "Briefly introduce the football games in the UK from 2004 to 2005."

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠦ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest 
pulling 04778965089b...   0% ▕▏    0 B/1.6 GB                  pulling manifest 
pulling 04778965089b...   0% ▕▏    0 B/1.6 GB                  pulling manifest 
pulling 04778965089b...   0% ▕▏    0 B/1.6 GB                  pulling manifest 
pulling 04778965089b...   0% ▕▏    0 B/1.6 GB                  pulling manifest 
pulling 04778965089b...   0% ▕▏    0 B/1.6 GB                  pulling manifest 
pulling 04778965089b...   0% ▕▏    0 B/1.6 GB                  pulling manifest 
pulling 04778965089b...   0% ▕▏    0 B/1.6 GB                  pulling manifest 
pulling 04778965089b...   0% ▕▏    0 B/1.6 GB                  pulling ma

Start to build our RAG. Firstly, this involves installing a bunch of packages:

In [4]:
# Install prerequisites
!pip install llama-index-embeddings-huggingface
!pip install llama-index-llms-ollama
!pip install llama-index-vector-stores-chroma
!pip install llama-index ipywidgets
!pip install llama-index-llms-huggingface
!pip install chromadb

# Import required modules from the llama_index library
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings
from llama_index.llms.ollama import Ollama
from llama_index.core import StorageContext

# Import ChromaVectorStore and chromadb module
from llama_index.vector_stores.chroma import ChromaVectorStore
import chromadb

# Import the Ollama class
from llama_index.llms.ollama import Ollama

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.4/15.4 MB 42.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 23.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 853.2/853.2 kB 57.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.7/320.7 kB 35.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 56.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 10.2 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
 

With everything installed we can setup (in Python) our LLM model:

In [5]:
# Use the global variable (OLLAMA_MODEL) as our LLM
# Set a timeout of 4 minutes
llm = Ollama(model=OLLAMA_MODEL, request_timeout=900.0)

Read in the text and implement the SetnenceSplitter

In [6]:
# Ensure the necessary packages are installed
!pip install llama-index

# Create a folder named 'data' in the current directory
### !mkdir -p data

from zipfile import ZipFile
import os
from llama_index.readers.file import FlatReader
from llama_index.core.node_parser import SentenceSplitter
from pathlib import Path # for finding the file



In [10]:
# Unzip the dataset
zip_file_path = '/content/bbcsport_football.zip'  # Update this path to the actual path of your zip file
unzip_dir = '/content'
with ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(unzip_dir)


In [9]:
! unzip('/content/bbcsport_football.zip')

/bin/bash: -c: line 1: syntax error near unexpected token `'/content/bbcsport_football.zip''
/bin/bash: -c: line 1: ` unzip('/content/bbcsport_football.zip')'


In [ ]:


# Combine the content of all text files into a single string
combined_text = ""
for filename in sorted(os.listdir(unzip_dir)):
    if filename.endswith(".txt"):
        with open(os.path.join(unzip_dir, filename), 'r', encoding='utf-8') as file:
            combined_text += file.read() + "\n"

# Write the combined text to a temporary file
temp_file_path = '/content/combined_bbcsport.txt'
with open(temp_file_path, 'w', encoding='utf-8') as temp_file:
    temp_file.write(combined_text)

bbcsport_docs = FlatReader().load_data(Path("/content/combined_bbcsport.txt"))

# we will limit to chunk size 100
parser = SentenceSplitter(chunk_size=200, chunk_overlap=100)
bbcsport_nodes = parser.get_nodes_from_documents(bbcsport_docs)

In [ ]:
bbcsport_nodes[1].text

IndexError: list index out of range

In [ ]:
# Ensure the necessary packages are installed
!pip install llama-index llama-index-embeddings-huggingface

# Create a folder named 'data' in the current directory
!mkdir -p data

from llama_index.readers.file import FlatReader
from llama_index.core.node_parser import SemanticSplitterNodeParser
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings
from pathlib import Path # for finding the file

# Load the data from the file
GOT_docs = FlatReader().load_data(Path("/content/GOTbook.txt"))

# Initialize a HuggingFace Embedding model
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")

# Specify the embedding model in LlamaIndex's settings
Settings.embed_model = embed_model

# Initialize the SemanticSplitterNodeParser with desired settings
parser = SemanticSplitterNodeParser(
    buffer_size=1,
    breakpoint_percentile_threshold=90,
    embed_model=embed_model
)

# Get semantic chunks from documents
GOT_nodes = parser.get_nodes_from_documents(GOT_docs)

# Iterate through the chunks and save each as a separate file
for count, node in enumerate(GOT_nodes):
    # Create a file name as a combination of the path ("/content/data/") ...
    # the name "Output", a number that increases with each chunk ...
    # and the file type (".txt").
    # e.g. the first file will be "/content/data/Output0.txt" ...
    # and the last will be "/content/data/OutputN.txt"
    fname = f"/content/data/Output{count}.txt"
    with open(fname, "w", encoding='utf-8') as text_file:
        text_file.write(node.text) # save the file

# Print the number of files created
print(f"Total files created: {len(GOT_nodes)}")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Total files created: 14489


Check the output

In [ ]:
GOT_nodes[0].text

'A Game Of Thrones \nBook One of A Song of Ice and Fire \nBy George R. R. Martin \nPROLOGUE \n"We should start back," Gared urged as the woods began to grow dark around them. "The wildlings are \ndead." \n'

In [ ]:
# Iterate through the chunks and save each as a separate file
for count, node in enumerate(GOT_nodes):
    # Create a file name as a combination of the path ("/content/data/") ...
    # the name "Output", a number that increases with each chunk ...
    # and the file type (".txt").
    # e.g. the first file will be "/content/data/Output0.txt" ...
    # and the last will be "/content/data/OutputN.txt"
    fname = f"/content/data/Output{count}.txt"
    with open(fname, "w", encoding='utf-8') as text_file:
        text_file.write(node.text) # save the file

# Print the number of files created
print(f"Total files created: {len(GOT_nodes)}")

Total files created: 14489


With all our documents created, we can check the first one to make sure everything worked:

(Pending)Now we will use Llama Index's SimpleDirectoryReader to load all the documents. After this we will specify an embedding model from HuggingFace and add all of this to Llama Index's settings:

In [ ]:
# Load documents
reader = SimpleDirectoryReader("/content/data") # load documents from the /data folder
docs = reader.load_data()
print(f"Loaded {len(docs)} docs")

# Initialize a HuggingFace Embedding model
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")

# Specify the LLM and embedding model into LlamaIndex's settings
Settings.llm = llm
Settings.embed_model = embed_model

Loaded 14489 docs


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


(Pending)Now we will setup a vector database (this time using Chroma DB) and index our documents (via embeddings). As in the previous tutorial, this will allow us to search for relevant documents to our query:

In [ ]:
# Create client ("db") and a database ("chroma_db")
db = chromadb.PersistentClient(path="./chroma_db")

# Create a collection/table ("demo-for-ram") in the db
chroma_collection = db.create_collection("demo-for-GOT")

# Set up ChromaVectorStore and load in data
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
# Specify Chroma as our vector db
storage_context = StorageContext.from_defaults(vector_store=vector_store)

# Create the index
index = VectorStoreIndex.from_documents(
    docs, # the abstract documents (e.g. "Output0.txt")
    storage_context = storage_context,
    embed_model = embed_model
)

# Print the metadata
print(chroma_collection)

# Print the name of the collection (table)
print(f'Collection name is: {chroma_collection.name}')

name='demo-for-GOT' id=UUID('f4c4973e-fdcd-4edd-a91e-9baa86daa2f8') metadata=None tenant='default_tenant' database='default_database'
Collection name is: demo-for-GOT


(Pending) Now everything is set up we can start (retrieval augmented) generating! Let's try a simple query about LLMs:

In [ ]:
query_engine = index.as_query_engine() # use the vector db for queries

response = query_engine.query("Describe the Battle of the Blackwater.") # query Phi-3 with context
response.response # print the response

' The Battle of the Blackwater is a conflict that took place in an unknown location. Based on the provided context information, it seems that the battle occurred during the War of the Roses, a series of wars between two rival families in medieval England who both claimed the throne as their own. While there isn\'t much information about the specifics of the battle, we can make some educated guesses based on the context clues.\n\nFirst, the mention of Blackwater suggests that it is a body of water that played a significant role in the conflict. It\'s possible that one of the families was trying to cross the Blackwater and were ambushed by the other family. Additionally, the reference to "hundreds" of ships burning might indicate that there was some kind of naval battle taking place as well.\n\nThe fact that the son of one of the families is at the Mud Gate with the Hand and the Kingsguard also suggests that there may have been a strategic military maneuvering going on. The hand and king

Inspect the output metadata

In [ ]:
response.metadata # print the full output from the LLM

{'60490dae-ea53-42e6-ab91-0b31e2bb8d19': {'file_path': '/content/data/Output7033.txt',
  'file_name': 'Output7033.txt',
  'file_type': 'text/plain',
  'file_size': 309,
  'creation_date': '2024-05-30',
  'last_modified_date': '2024-05-30'},
 '42e47982-bb04-48be-991e-f2f8069d43b8': {'file_path': '/content/data/Output8055.txt',
  'file_name': 'Output8055.txt',
  'file_type': 'text/plain',
  'file_size': 261,
  'creation_date': '2024-05-30',
  'last_modified_date': '2024-05-30'}}